In [1]:
import pandas as pd
import numpy as np
import os
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
dir_path = './' #local
# dir_path = './drive/MyDrive/' #colab

In [2]:
#uncomment in colab
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
files = os.listdir(dir_path + 'Merged')
files

['gestures_test.csv', 'gestures_train.csv']

In [4]:
df_train = pd.read_csv(dir_path + 'Merged/gestures_train.csv')
df_test = pd.read_csv(dir_path + 'Merged/gestures_test.csv')
df_train

,label,a1,a2,ab,b1,b2,bc,c1,c2,cd,...,y0,y1,y2,y3,y4,z0,z1,z2,z3,z4
0,1,171.917694,132.738831,116.083084,168.437363,169.912949,2.744997,95.554222,125.426796,1.433788,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,172.026489,132.672134,116.079544,168.437363,169.912949,0.662870,95.559898,125.403061,2.515430,...,41.752968,0.023743,0.069817,0.022484,0.015427,-60.686092,0.031288,-0.019450,0.028371,0.025351
2,1,172.049591,132.658401,116.078300,168.437317,169.913010,1.706766,95.556076,125.403023,1.034447,...,0.004364,0.005936,0.000786,0.005386,0.003067,0.007065,0.008162,0.010556,0.006170,0.006201
3,1,172.071579,132.642776,116.077629,168.437347,169.912949,0.455037,95.549484,125.403069,0.707033,...,0.004433,0.005936,-0.003082,0.005371,0.004326,0.007211,0.008144,0.013555,0.006168,0.004779
4,1,172.092804,132.627594,116.077316,168.437347,169.912933,2.034597,95.544991,125.404495,1.652241,...,0.004295,0.002213,-0.005486,0.002556,0.000992,0.004589,0.001745,0.009327,0.003395,0.003071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161795,30,155.428940,174.586319,110.989807,168.322968,170.471313,0.891064,100.356743,141.077988,0.197881,...,0.300800,0.270683,0.573120,0.451969,0.419716,-0.081818,-0.191807,-0.038231,-0.073334,-0.167938
161796,30,155.546890,175.031281,111.164673,168.517853,170.631866,1.755110,100.357552,141.073151,0.849069,...,-0.127975,0.304504,-0.441071,-0.423367,-0.339947,-0.481194,-0.374904,-0.335571,-0.306252,-0.319984
161797,30,155.883240,175.916321,111.659500,168.709732,170.790070,0.870190,100.386345,141.060516,1.251321,...,-0.388813,-0.135117,-1.058586,-0.907379,-0.691280,-0.401161,0.119175,-0.086823,-0.160332,-0.248985
161798,30,156.244843,176.664246,112.280891,168.823837,170.889343,1.974926,100.382736,141.060654,0.438103,...,-0.584534,1.845512,-2.698456,-2.489361,-1.933666,-3.416515,-1.572324,-2.387756,-2.609138,-2.599831


In [5]:
# x = df_train.values #returns a numpy array
# min_max_scaler = MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(x)
# df_train = pd.DataFrame(x_scaled)

# x = df_test.values #returns a numpy array
# min_max_scaler = MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(x)
# df_test = pd.DataFrame(x_scaled)

In [6]:
def calculation(df_rows):
    res = {'omega0': [],
           'omega1': [],
           'omega2': [],
           'omega3': [],
           'omega4': [],
           'beta0': [],
           'beta1': [],
           'beta2': [],
           'beta3': [],
           'beta4': [],
           'u0':[],
           'u1':[],
           'u2':[],
           'u3':[],
           'u4':[],
           'u5':[],
           'v0':[],
           'v1':[],
           'v2':[],
           'v3':[],
           'v4':[],
           'v5':[],
           'z0':[],
           'z1':[],
           'z2':[],
           'z3':[],
           'z4':[],
           'z5':[],
           'gamma3': [],
           'gamma2': [],
           'gamma1': []
          }
    for index, row in df_rows.iterrows():
        #intra-finger angles
        res['gamma3'].append(row['ab'])
        res['gamma2'].append(row['bc'])
        res['gamma1'].append(row['cd'])
        
        #finger tip coordinates
        res['u0'].append(row['x0'])
        res['u1'].append(row['x1'])
        res['u2'].append(row['x2'])
        res['u3'].append(row['x3'])
        res['u4'].append(row['x4'])
        
        res['v0'].append(row['y0'])
        res['v1'].append(row['y1'])
        res['v2'].append(row['y2'])
        res['v3'].append(row['y3'])
        res['v4'].append(row['y4'])
        
        res['z0'].append(row['z0'])
        res['z1'].append(row['z1'])
        res['z2'].append(row['z2'])
        res['z3'].append(row['z3'])
        res['z4'].append(row['z4'])
        
        #palm coordinates
        res['u5'].append(row['px'])
        res['v5'].append(row['py'])
        res['z5'].append(row['pz'])
        
        #joint angles omega
        res['omega0'].append(row['e1'])
        res['omega1'].append(row['d1'])
        res['omega2'].append(row['c1'])
        res['omega3'].append(row['b1'])
        res['omega4'].append(row['a1'])
        
        #joint angles beta
        res['beta0'].append(row['e2'])
        res['beta1'].append(row['d2'])
        res['beta2'].append(row['c2'])
        res['beta3'].append(row['b2'])
        res['beta4'].append(row['a2'])
#     x = pd.DataFrame(res).values #returns a numpy array
#     min_max_scaler = MinMaxScaler()
#     x_scaled = min_max_scaler.fit_transform(x)
#     res = pd.DataFrame(x_scaled)
    res = pd.DataFrame(res)
#     display(res)
    return res.to_numpy()

In [7]:
def scale_data(data, min_max_scaler):
    for i in range(len(data)):
        data[i] = min_max_scaler.transform(data[i])
    return data

def map_features(df):
    output_df = []
    output_label = []

    for i in range(200,len(df)+1,200):
        if i == 200:
            output_df.append(np.copy(calculation(df[:i])))
            output_label.append(df[:i]['label'][i-1]-1)
        else:
            output_df.append(np.copy(calculation(df[(i-200):i])))
            output_label.append(df[(i-200):i]['label'][i-1]-1)
    return np.array(output_df), np.array(output_label)


    

In [8]:
min_max_scaler = MinMaxScaler(feature_range=(0,1))
x_train, y_train = map_features(df_train)

num_instances, num_time_steps, num_features = x_train.shape
x_train = np.reshape(x_train, newshape=(-1, num_features))
x_train = min_max_scaler.fit_transform(x_train)
x_train = np.reshape(x_train, newshape=(num_instances, num_time_steps, num_features))

x_train, y_train = shuffle(x_train, y_train, random_state=0)

x_test, y_test = map_features(df_test)
num_instances, num_time_steps, num_features = x_test.shape
x_test = np.reshape(x_test, newshape=(-1, num_features))
x_test = min_max_scaler.transform(x_test)
x_test = np.reshape(x_test, newshape=(num_instances, num_time_steps, num_features))

x_test, y_test = shuffle(x_test, y_test, random_state=0)


In [9]:
# x_train = np.delete(x_train, [0,1,2,3,4,5,6,7,8], 0)
# y_train = np.delete(y_train, [0,1,2,3,4,5,6,7,8], 0)
print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)
# print(y_train)
# print("\n")
# print(y_test)

# print(x_train)
y_train = y_train.astype('int')

# print(y_train)


x_train.shape:  (809, 200, 31)
y_train.shiape:  (809,)
x_test.shape:  (389, 200, 31)
y_test.shape:  (389,)


In [14]:
# np.set_printoptions(threshold=np.inf)
display(x_test[0])


array([[9.70850986e-01, 9.70940637e-01, 9.46023480e-01, 9.51898438e-01,
        8.22819279e-01, 9.62321427e-01, 9.59429609e-01, 9.29404428e-01,
        9.53618328e-01, 6.37061186e-01, 5.05414320e-01, 5.38790197e-01,
        5.33239335e-01, 5.11299808e-01, 4.82183522e-01, 4.01623206e-01,
        4.61403560e-01, 4.66579692e-01, 5.83377323e-01, 5.82960056e-01,
        5.80651403e-01, 5.24261618e-01, 6.81945684e-01, 4.00065453e-01,
        3.78867684e-01, 3.61267178e-01, 3.55296494e-01, 4.61144730e-01,
        5.31501046e-01, 1.19338591e-02, 4.18376240e-02],
       [9.70921301e-01, 9.71807973e-01, 9.46386307e-01, 9.51756585e-01,
        8.23415608e-01, 9.62396307e-01, 9.60435665e-01, 9.29828039e-01,
        9.53448571e-01, 6.35924657e-01, 8.96488246e-01, 5.38676190e-01,
        5.33162373e-01, 5.11146211e-01, 4.81967483e-01, 4.01161553e-01,
        5.98298925e-01, 4.66903102e-01, 5.83789371e-01, 5.84932481e-01,
        5.80849265e-01, 5.24639779e-01, 3.23869152e-01, 4.00170974e-01,
       

In [ ]:
model = Sequential()
model.add(LSTM(units=200, input_shape=x_train.shape[1:], return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(200, return_sequences=True ))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(200, return_sequences=True ))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(200))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# model.add(Dense(30, activation='softmax'))

model.add(Dense(30, activation='softmax'))
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_24 (LSTM)               (None, 200, 200)          185600    
_________________________________________________________________
batch_normalization_24 (Batc (None, 200, 200)          800       
_________________________________________________________________
dropout_24 (Dropout)         (None, 200, 200)          0         
_________________________________________________________________
lstm_25 (LSTM)               (None, 200, 200)          320800    
_________________________________________________________________
batch_normalization_25 (Batc (None, 200, 200)          800       
_________________________________________________________________
dropout_25 (Dropout)         (None, 200, 200)          0         
_________________________________________________________________
lstm_26 (LSTM)               (None, 200, 200)         

In [ ]:
opt = tf.keras.optimizers.Adam(lr=0.0001)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)

gestures = model.fit(x = x_train,
          y = y_train,
          epochs=800,
          validation_split=0.1, #split 10% of the trainning set for the validation set,
          batch_size=80,
#           validation_data = (x_test, y_test),
#           shuffle=True
         )

Epoch 1/800
10/10 [==============================] - 7s 222ms/step - loss: 3.7425 - accuracy: 0.1052 - val_loss: 3.3891 - val_accuracy: 0.0000e+00
Epoch 2/800
10/10 [==============================] - 1s 96ms/step - loss: 1.9315 - accuracy: 0.4586 - val_loss: 3.3725 - val_accuracy: 0.0000e+00
Epoch 3/800
10/10 [==============================] - 1s 95ms/step - loss: 1.3303 - accuracy: 0.6428 - val_loss: 3.3576 - val_accuracy: 0.0000e+00
Epoch 4/800
10/10 [==============================] - 1s 95ms/step - loss: 1.1345 - accuracy: 0.6745 - val_loss: 3.3437 - val_accuracy: 0.0000e+00
Epoch 5/800
10/10 [==============================] - 1s 95ms/step - loss: 0.9273 - accuracy: 0.7720 - val_loss: 3.3310 - val_accuracy: 0.0247
Epoch 6/800
10/10 [==============================] - 1s 95ms/step - loss: 0.8639 - accuracy: 0.7794 - val_loss: 3.3141 - val_accuracy: 0.0247
Epoch 7/800
10/10 [==============================] - 1s 95ms/step - loss: 0.8406 - accuracy: 0.7615 - val_loss: 3.2894 - val_accura

In [ ]:
model.save(dir_path + 'Models/gestures.h5')

In [ ]:
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=80)
print("test loss, test acc:", results)

Evaluate on test data
5/5 [==============================] - 0s 58ms/step - loss: 0.9486 - accuracy: 0.8149
test loss, test acc: [0.9486066102981567, 0.8149100542068481]


In [15]:
new_model = load_model(dir_path + 'Models/gestures.h5')
print("Evaluate on test data")
results = new_model.evaluate(x_test, y_test, batch_size=80)
print("test loss, test acc:", results)

Evaluate on test data
5/5 [==============================] - 5s 75ms/step - loss: 0.9486 - accuracy: 0.8149
test loss, test acc: [0.9486066102981567, 0.8149100542068481]


In [16]:
new_model.save('Models/gestures', save_format='tf')

INFO:tensorflow:Assets written to: Models/gestures\assets


INFO:tensorflow:Assets written to: Models/gestures\assets
